In [ ]:
from utils import *

sda = spatialDA()
sda.check_torch_gpu()

octave = sda.mrst_startup()

In [ ]:
# Grid
dims = 64
nx, ny, nz = dims, dims, 1
dx, dy, dz = 1000*meter(), 1000*meter(), 100*meter()

G = octave.feval('cartGrid', [nx,ny,nz], [dx,dy,dz])
G = octave.feval('computeGeometry', G, store_as='G')

In [ ]:
from gstools import SRF, TPLStable
x = y = np.linspace(0, 64, 64)
model = TPLStable(dim       = 2,
                  var       = 1,
                  len_low   = 0,
                  len_scale = 16,
                  nugget    = 0.1,
                  anis      = 0.5,
                  angles    = np.pi/4,
                  alpha     = 1.5,
                  hurst     = 0.7)
srf = SRF(model, mean=1, mode_no=1000, seed=2023, verbose=False)
logperm = srf((x,y), mesh_type='structured')

In [ ]:
# Rock Properties
G = octave.pull('G')
perm = (logperm*milli()*darcy()).flatten().reshape(-1,1)
poro = (10**((logperm-7)/10)).flatten().reshape(-1,1)
rock = octave.feval('makeRock', G, perm, poro)

In [ ]:
# Initial State
rhow = 1000
P0   = 4000*psia()
initState = {}
initState['pressure'] = repmat(P0,    int(G.cells.num), 1)
initState['s']        = repmat([1,0], int(G.cells.num), 1)
initState['sGmax']    = initState['s'][:,1]
octave.push('initState', initState)

In [ ]:
# Fluid Properties
octave.feval('CO2props', store_as='co2')

octave.push('p_ref', 30*mega()*Pascal())
octave.push('t_ref', 94 + 273.15)

octave.push('rhoc',    octave.eval('co2.rho(p_ref,t_ref)', verbose=False))
octave.push('rhow',    1000)

octave.push('cf_co2',  octave.eval('co2.rhoDP(p_ref,t_ref)', verbose=False))
octave.push('cf_wat',  0)
octave.push('cf_rock', 4.35e-5 / barsa())

octave.push('muw',     8e-4 * Pascal() * second())
octave.push('muco2', octave.eval('co2.mu(p_ref,t_ref)', verbose=False) * Pascal() * second())

octave.push('fluid', octave.feval("initSimpleADIFluid('phases', 'WG',            \
                                                     'mu',     [muw, muco2],    \
                                                     'rho',    [rhow, rhoc],    \
                                                     'pRef',   p_ref,           \
                                                     'c',      [cf_wat, cf_co2],\
                                                     'cR',     cf_rock,         \
                                                     'n',      [2,2])", verbose=False))

srw, src = 0.27, 0.20

In [ ]:
octave.pull('fluid')